# One to One

In [2]:
import torch
import numpy as np
import gc

device = 'mps'

# RNN

In [9]:
class SimpleRNN(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.RNN(input_size=self.I,
                                hidden_size=self.H,
                                nonlinearity='tanh',
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0 = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])

        return out

# LSTM

In [11]:
class LSTM(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.LSTM(input_size=self.I,
                                 hidden_size=self.H,
                                 batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)
        c0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, (h_0, c_0) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

# GRU

In [12]:
class GRU(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.GRU(input_size=self.I,
                                hidden_size=self.H,
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0= self.rnn(x, h0)
        out = self.fc(out[:, -1, :])

        return out

# 모델 생성

In [32]:
rnn_model = SimpleRNN(n_input=1, n_hidden=40, n_output=1).to(device)
lstm_model = LSTM(n_input=1, n_hidden=40, n_output=1).to(device)
gru_model = GRU(n_input=1, n_hidden=40, n_output=1).to(device)

# 학습

In [33]:
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 랜덤한 값을 생성 -> 1~4 사이의 값을 1000개 생성
y = np.square(X) # x의 제곱

train_X = torch.from_numpy(X.astype(np.float32)).to(device)
train_y = torch.from_numpy(y.astype(np.float32)).squeeze(-1).to(device)

crietrion = torch.nn.MSELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters())
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
gru_optimizer = torch.optim.Adam(gru_model.parameters())

In [20]:
for epoch in range(4000):
    outputs = rnn_model(train_X)
    loss = crietrion(outputs, train_y)

    rnn_optimizer.zero_grad()
    loss.backward()
    rnn_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 56.4717
Epoch [200/4000], Loss: 27.7995
Epoch [300/4000], Loss: 20.3089
Epoch [400/4000], Loss: 15.2038
Epoch [500/4000], Loss: 9.8030
Epoch [600/4000], Loss: 5.6479
Epoch [700/4000], Loss: 3.4947
Epoch [800/4000], Loss: 2.6940
Epoch [900/4000], Loss: 2.3929
Epoch [1000/4000], Loss: 2.2170
Epoch [1100/4000], Loss: 2.0728
Epoch [1200/4000], Loss: 1.9389
Epoch [1300/4000], Loss: 1.8087
Epoch [1400/4000], Loss: 1.6789
Epoch [1500/4000], Loss: 1.5481
Epoch [1600/4000], Loss: 1.4157
Epoch [1700/4000], Loss: 1.2818
Epoch [1800/4000], Loss: 1.1472
Epoch [1900/4000], Loss: 1.0133
Epoch [2000/4000], Loss: 0.8819
Epoch [2100/4000], Loss: 0.7553
Epoch [2200/4000], Loss: 0.6357
Epoch [2300/4000], Loss: 0.5253
Epoch [2400/4000], Loss: 0.4258
Epoch [2500/4000], Loss: 0.3383
Epoch [2600/4000], Loss: 0.2632
Epoch [2700/4000], Loss: 0.2005
Epoch [2800/4000], Loss: 0.1493
Epoch [2900/4000], Loss: 0.1087
Epoch [3000/4000], Loss: 0.0773
Epoch [3100/4000], Loss: 0.0536
Epoch [3200/4

In [21]:
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device)
print(f"Input: 2.0, Output: {rnn_model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Input: 2.0, Output: 4.022846221923828, 정답: 4.0


In [22]:
for epoch in range(4000):
    outputs = lstm_model(train_X)
    loss = crietrion(outputs, train_y)

    lstm_optimizer.zero_grad()
    loss.backward()
    lstm_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 71.2747
Epoch [200/4000], Loss: 36.1348
Epoch [300/4000], Loss: 19.2146
Epoch [400/4000], Loss: 13.1781
Epoch [500/4000], Loss: 7.5683
Epoch [600/4000], Loss: 3.9404
Epoch [700/4000], Loss: 2.5295
Epoch [800/4000], Loss: 2.0122
Epoch [900/4000], Loss: 1.6879
Epoch [1000/4000], Loss: 1.4118
Epoch [1100/4000], Loss: 1.1594
Epoch [1200/4000], Loss: 0.9278
Epoch [1300/4000], Loss: 0.7199
Epoch [1400/4000], Loss: 0.5396
Epoch [1500/4000], Loss: 0.3899
Epoch [1600/4000], Loss: 0.2714
Epoch [1700/4000], Loss: 0.1823
Epoch [1800/4000], Loss: 0.1188
Epoch [1900/4000], Loss: 0.0757
Epoch [2000/4000], Loss: 0.0480
Epoch [2100/4000], Loss: 0.0307
Epoch [2200/4000], Loss: 0.0203
Epoch [2300/4000], Loss: 0.0140
Epoch [2400/4000], Loss: 0.0101
Epoch [2500/4000], Loss: 0.0076
Epoch [2600/4000], Loss: 0.0059
Epoch [2700/4000], Loss: 0.0046
Epoch [2800/4000], Loss: 0.0037
Epoch [2900/4000], Loss: 0.0029
Epoch [3000/4000], Loss: 0.0023
Epoch [3100/4000], Loss: 0.0018
Epoch [3200/4

In [23]:
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device)
print(f"Input: 2.0, Output: {lstm_model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Input: 2.0, Output: 3.983644485473633, 정답: 4.0


In [34]:
for epoch in range(4000):
    outputs = gru_model(train_X)
    loss = crietrion(outputs, train_y)

    gru_optimizer.zero_grad()
    loss.backward()
    gru_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 56.0298
Epoch [200/4000], Loss: 28.9062
Epoch [300/4000], Loss: 19.7091
Epoch [400/4000], Loss: 13.6193
Epoch [500/4000], Loss: 10.0904
Epoch [600/4000], Loss: 7.2915
Epoch [700/4000], Loss: 5.3637
Epoch [800/4000], Loss: 4.6592
Epoch [900/4000], Loss: 4.3629
Epoch [1000/4000], Loss: 4.1304
Epoch [1100/4000], Loss: 3.8627
Epoch [1200/4000], Loss: 3.6502
Epoch [1300/4000], Loss: 4.1701
Epoch [1400/4000], Loss: 4.8456
Epoch [1500/4000], Loss: 3.2514
Epoch [1600/4000], Loss: 2.1388
Epoch [1700/4000], Loss: 1.5047
Epoch [1800/4000], Loss: 1.1339
Epoch [1900/4000], Loss: 0.9171
Epoch [2000/4000], Loss: 0.7871
Epoch [2100/4000], Loss: 0.7243
Epoch [2200/4000], Loss: 0.6824
Epoch [2300/4000], Loss: 0.6458
Epoch [2400/4000], Loss: 0.6108
Epoch [2500/4000], Loss: 0.5801
Epoch [2600/4000], Loss: 0.5625
Epoch [2700/4000], Loss: 0.5539
Epoch [2800/4000], Loss: 0.5485
Epoch [2900/4000], Loss: 0.5731
Epoch [3000/4000], Loss: 0.5603
Epoch [3100/4000], Loss: 0.5402
Epoch [3200/

In [35]:
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device)
print(f"Input: 2.0, Output: {gru_model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Input: 2.0, Output: 4.410461902618408, 정답: 4.0
